<a href="https://colab.research.google.com/github/steysie/RussianNumberConverter/blob/master/RussianNumberConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Russian Number Converter
#### Anastasia Nikiforova

Задача - конвертировать имена числительные в предложении в текстовый эквивалент с созранением согласования с другими членами предложения.

Пример: 

*Я прочитал сказку об 1 маленькой черепашке и 127 гномах --> Я прочитал сказку об одной черепашке и ста двадцати семи гномах*

*К 7 часам в кассе уже нет 1928 билетов --> К семи часам в кассе уже нет одной тысячи девытисот двадцати восьми билетов*



In [0]:
!pip install num2words

In [0]:
!pip install pymorphy2

In [0]:
import re
import pymorphy2
from spacy.lang.ru import Russian
from num2words import num2words
import pandas as pd
import numpy as np

morph = pymorphy2.MorphAnalyzer()

### Об использованных библиотеках

```num2words``` - готовая библиотека для конвертации

Сам по себе модуль не решает задачу, но его можно использовать для конвертации найденных чисел в развернутый формат. После такой конвертации будет необходимо изменить окончания в зависимости от грамматической формы зависимого слова.

In [4]:
# пример работы
print(f'cardinal 14263451234 = {num2words(1426345231234, lang="ru")}')
print(f'ordinal 14234 = {num2words(14234, to="ordinal", lang="ru")}')

cardinal 14263451234 = один триллион четыреста двадцать шесть миллиардов триста сорок пять миллионов двести тридцать одна тысяча двести тридцать четыре
ordinal 14234 = четырнадцать тысяч двести тридцать четвертый


```pymorphy2``` - включает в себя размеченный Open Corpora, в котором содержатся такие необходимые грамматические категории как род, падеж и одушевленность.

In [5]:
# пример работы
word = "ежами"
num = "шестнадцать"

word_tag = morph.parse(word)[0]
num_tag = morph.parse(num)[0]

new_num = num_tag.inflect({word_tag.tag.case}).word
f"{new_num} {word}"

'шестнадцатью ежами'

## Конвертация чисел в слова

In [0]:
class RussianNumberConverter:
    '''
    The Russian Number Converter inputs and outputs a string.
    It is fully rule-based, and uses annotated Open Corpora via pymorphy2.
    Rules can be modified and supplemented if needed. Existing rules cover most rules of number agreement in Rusian language.
    It is suggested to feed sentence-length segments to the converter.

    Example input:  "2 больших капибар"
    Example output: "двух больших капибар"
    '''

    def __init__(self):
        self.new_string = ""

    def tokenize(self, text):
        ''' 
        Spacy tokenizer.
        Each word and puctuation mark is a separate token.
        '''
        nlp = Russian()
        doc = nlp(text)

        return [token.text for token in doc]

    def convert(self, text):

        tmp = self.tokenize(text)

        for tok_n in range(len(tmp)):
            if re.match(r"\d+", tmp[tok_n]):
                raw_num = num2words(tmp[tok_n], lang="ru")
                
                # результат конвертации может быть составным числительным 
                raw_num_split = raw_num.split()
                raw_num_tagged = [morph.parse(i)[0] for i in raw_num_split]

                if tok_n>0 and tmp[tok_n-1].lower() == "нет":

                    # поставить числительное в форму Родительного падежа
                    raw_num_split = [token.inflect({"gent"}).word for token in raw_num_tagged]

                elif tok_n>0 and tmp[tok_n-1].lower() == "c":

                    # поставить числительное в форму Предложного падежа
                    raw_num_split = [token.inflect({"ablt"}).word for token in raw_num_tagged]

                # если за словом следует имя существительное, прилагательное или причастие - согласуем с ним числительное

                elif tok_n != len(tmp)-1 and any(tag in [morph.parse(i)[0].tag.POS for i in tmp[tok_n+1:]] for tag in ["NOUN", "ADJF"]):
                    
                    # согласовать с их родом и числом (inflect)
                    next_tok = next(morph.parse(tok)[0] for tok in tmp[tok_n+1:] if morph.parse(tok)[0].tag.POS in ["NOUN", "ADJF"])
                    
                    if raw_num_split[-1] in ["один", "два"]:
                        # для чисел, оканцивающихся на "один" и "два" меняем только последнее слово

                        # если зависимое слово - существительное, согласуем по роду и падежу
                        if morph.parse(tmp[tok_n+1])[0].tag.POS == "NOUN":
                            # смотрим на одушевленность
                            if next_tok.tag.animacy == "inan":
                                if next_tok.tag.gender:
                                    raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.gender, next_tok.tag.case}).word

                                if raw_num_tagged[-1].tag.animacy:
                                    raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.animacy, next_tok.tag.case}).word
                                
                                elif next_tok.tag.case == "gent":
                                    if next_tok.tag.gender:
                                        raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.gender, "accs"}).word
                                    else:
                                        raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.animacy, "accs"}).word

                                else:
                                    raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.case}).word
                            
                            else:
                                raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.gender, next_tok.tag.case}).word
                        
                        # если зависимое слово - прилагательное, согласуем по падежу
                        if next_tok.tag.POS in ["ADJF", "PRTF"]:
                            if next_tok.tag.gender:
                                raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.gender, next_tok.tag.case}).word

                            else:
                                raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.case}).word

                    #### noun == "gent" --> numr == "accs"
                    elif next_tok.tag.case == "gent":
                        raw_num_split = [token.inflect({'accs'}).word for token in raw_num_tagged]

                    else:     
                        raw_num_split = [token.inflect({next_tok.tag.case}).word for token in raw_num_tagged]

                elif tok_n>0 and morph.parse(tmp[tok_n-1])[0].tag.POS == "PREP":

                    # поставить числительное в форму Предложного падежа
                    raw_num_split = [token.inflect({"loct"}).word for token in raw_num_tagged]

                # если предыдущие условия не подходят, оставляем как есть - предполагаем, что таких случаев незначительно мало и именительный падеж подходит
                accord_number = " ".join([i for i in raw_num_split])
                tmp[tok_n] = accord_number
        
        self.new_string = " ".join(text for text in tmp)

        return self.new_string


In [0]:
conv = RussianNumberConverter()

In [31]:
s = ["5 котов",
     "1 котом",
     "10 котами",
     "2 больших капибар",
     "о 55 гномах",
     "41 чертенку",
     "1 маленькой черепашки",
     "2 стола", 
     "18 мест",
     "нет 18 мест",
     "нет 1928 билетов",
     "8 столами",
     "9 котами", 
     "С 274 карандашами",
     "О 2", 
     "главный приз вручили 1 танцующему дельфину"]

[conv.convert(i) for i in s]

['пять котов',
 'одним котом',
 'десятью котами',
 'двух больших капибар',
 'о пятидесяти пяти гномах',
 'сорок одному чертенку',
 'одной маленькой черепашки',
 'два стола',
 'восемнадцать мест',
 'нет восемнадцати мест',
 'нет одной тысячи девятисот двадцати восьми билетов',
 'восемью столами',
 'девятью котами',
 'С двумястами семьюдесятью четырьмя карандашами',
 'О двух',
 'главный приз вручили одному танцующему дельфину',
 'я никогда не видел тридцать шесть танцующих дельфинов']